In [55]:
from deep_translator import GoogleTranslator

# Function to translate text using deep_translator
def deep_translate_text(text, target='en'):
    translator = GoogleTranslator(source='auto', target=target)
    return translator.translate(text)

In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setting up the Chrome WebDriver
options = webdriver.ChromeOptions()
options.headless = True  # Set headless mode to True to run without opening a browser window
driver = webdriver.Chrome(options=options)

data_inc = {'car_name': [], 'car_info': [], 'car_year_hand': [], 'car_price': [],'car_tag': []}
for i in range(1, 151):
    url = f"https://www.yad2.co.il/vehicles/cars?page={i}"
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Simple condition
    except TimeoutException:
        print("Timed out waiting for page to load: ", url)
        continue
    
    # Scroll to the bottom of the page (if needed)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow extra time for any lazy-loaded elements
    
    # Extract HTML content
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    car_boxes = soup.find_all('div', class_='feed-item-base_feedItemBox__5WVY1 scale-up-on-hover_scaleUpOnHover__QSVDw')
    for box in car_boxes:
        car_name = box.find('span', class_='feed-item-info_heading__k5pVC').get_text(strip=True)
        car_info = box.find('span', class_='feed-item-info_marketingText__eNE4R').get_text(strip=True)
        car_year_hand = box.find('span', class_='feed-item-info_yearAndHandBox___JLbc').get_text(strip=True)
        car_price = box.find('span', class_='price_price__xQt90').get_text(strip=True)
        car_tags = box.find('div', class_='tag-list_upgradeItemFlagBox__qPadL')
        if car_tags:
            car_tags_texts = [tag.get_text(strip=True) for tag in car_tags.find_all('span', class_='tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj')]
        else:
            car_tags_texts = ['None','Data']
        # data_inc['car_tag'].extend(car_tags_texts)
        data_inc['car_name'].append(car_name)
        data_inc['car_info'].append(car_info)
        data_inc['car_year_hand'].append(car_year_hand)
        data_inc['car_price'].append(car_price)
        data_inc['car_tag'].append(','.join(car_tags_texts))
        # if title_element and date_element:
        #     title = title_element.get_text(strip=True)
        #     date = date_element.get_text(strip=True)
        #     link = title_element.find('a')['href'] if title_element.find('a') else None
        #     data_inc['title'].append(title)
        #     data_inc['date'].append(date)
        #     data_inc['link'].append(link)
# Close the driver after the loop
driver.quit()# Display the first few rows of the DataFrame


KeyboardInterrupt: 

In [57]:
# Convert the collected data to a DataFrame
df_car_prices = pd.DataFrame(data_inc)
df_car_prices


,car_name,car_info,car_year_hand,car_price,car_tag
0,ג'יפ / Jeep רנגלר ארוך,4X4 Unlimited Sahara אוט' 5 דל' 2.0 (272 כ''ס),2021 • יד 1,"300,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה"
1,אאודי Q7,4X4 Premium חשמלי דיזל אוט' 5 מק' 3.0 (374 כ''ס),2018 • יד 2,"216,000 ₪","חסכוני,היברידי,טסט בתוקף"
2,סובארו פורסטר,4X4 B אוט' 2.0 (150 כ''ס) [2008-2011],2011 • יד 3,"42,000 ₪","שטח קשוח,טסט בתוקף,מערכת מולטימדיה"
3,ג'יפ / Jeep גרנד צ'ירוקי,4X4 Laredo אוט' 5 מק' 3.6 (286 כ''ס),2019 • יד 1,"160,000 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף"
4,טויוטה לנד קרוזר ארוך,"4X4 STD אוט' דיזל 5 מק' 3.0 (173 כ""ס)",2008 • יד 1,"123,000 ₪","מיכל דלק גדול,טסט בתוקף,יד ראשונה"
...,...,...,...,...,...
8475,סקודה קודיאק,Style אוט' דיזל 7 מק' 4 דל' 2.0 (150 כ''ס),2018 • יד 1,"10,000 ₪","טסט בתוקף,יד ראשונה"
8476,קיה נירו הייבריד,LX הייבריד אוט' 1.6 (141 כ''ס),2019 • יד 1,"73,900 ₪","בקרת שמירת מרחק,גלגלי מגנזיום"
8477,סיטרואן C4 ספייסטורר,Grand Exclus BlueHdi אוט' דיזל 7 מק' 1.5(130 כ...,2020 • יד 1,"6,500 ₪","בקרת שמירת מרחק,גלגלי מגנזיום"
8478,טויוטה קורולה הייבריד,Sun סדאן אוט' 1.8 (98 כ''ס),2020 • יד 1,"98,900 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק"


In [58]:
df_car_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8480 entries, 0 to 8479
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   car_name       8480 non-null   object
 1   car_info       8480 non-null   object
 2   car_year_hand  8480 non-null   object
 3   car_price      8480 non-null   object
 4   car_tag        8480 non-null   object
dtypes: object(5)
memory usage: 331.4+ KB


In [62]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='en')

# Function to translate text using deep_translator
def deep_translate_text(text, target='en'):
    return translator.translate(text)

# Apply the translation function to the DataFrame
df_car_prices['car_name_en'] = df_car_prices['car_name'].apply(deep_translate_text)
# df_car_prices['car_info_en'] = df_car_prices['car_info'].apply(deep_translate_text)
# df_car_prices['car_year_hand_en'] = df_car_prices['car_year_hand'].apply(deep_translate_text)
# df_car_prices['car_price_en'] = df_car_prices['car_price'].apply(deep_translate_text)
# df_car_prices['car_tags_en'] = df_car_prices['car_tag'].apply(deep_translate_text)

df_car_prices

KeyboardInterrupt: 

In [63]:
df_car_prices

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en
0,ג'יפ / Jeep רנגלר ארוך,4X4 Unlimited Sahara אוט' 5 דל' 2.0 (272 כ''ס),2021 • יד 1,"300,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Jeep / Jeep Wrangler long
1,אאודי Q7,4X4 Premium חשמלי דיזל אוט' 5 מק' 3.0 (374 כ''ס),2018 • יד 2,"216,000 ₪","חסכוני,היברידי,טסט בתוקף",Audi Q7
2,סובארו פורסטר,4X4 B אוט' 2.0 (150 כ''ס) [2008-2011],2011 • יד 3,"42,000 ₪","שטח קשוח,טסט בתוקף,מערכת מולטימדיה",Subaru Forester
3,ג'יפ / Jeep גרנד צ'ירוקי,4X4 Laredo אוט' 5 מק' 3.6 (286 כ''ס),2019 • יד 1,"160,000 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף",Jeep / Jeep Grand Cherokee
4,טויוטה לנד קרוזר ארוך,"4X4 STD אוט' דיזל 5 מק' 3.0 (173 כ""ס)",2008 • יד 1,"123,000 ₪","מיכל דלק גדול,טסט בתוקף,יד ראשונה",Toyota Land Cruiser long
...,...,...,...,...,...,...
8475,סקודה קודיאק,Style אוט' דיזל 7 מק' 4 דל' 2.0 (150 כ''ס),2018 • יד 1,"10,000 ₪","טסט בתוקף,יד ראשונה",Skoda Kodiaq
8476,קיה נירו הייבריד,LX הייבריד אוט' 1.6 (141 כ''ס),2019 • יד 1,"73,900 ₪","בקרת שמירת מרחק,גלגלי מגנזיום",Kia Niro Hybrid
8477,סיטרואן C4 ספייסטורר,Grand Exclus BlueHdi אוט' דיזל 7 מק' 1.5(130 כ...,2020 • יד 1,"6,500 ₪","בקרת שמירת מרחק,גלגלי מגנזיום",Citroen C4 Speedorer
8478,טויוטה קורולה הייבריד,Sun סדאן אוט' 1.8 (98 כ''ס),2020 • יד 1,"98,900 ₪","בקרת שיוט אדפטיבית,בקרת שמירת מרחק",Toyota Corolla Hybrid


In [83]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='en')

# Function to translate text using deep_translator
def deep_translate_text(text, target='en'):
    return translator.translate(text)

# Apply the translation function to the DataFrame
df_car_prices['car_name_en'] = df_car_prices['car_name'].apply(deep_translate_text)
# df_car_prices['car_info_en'] = df_car_prices['car_info'].apply(deep_translate_text)
# df_car_prices['car_year_hand_en'] = df_car_prices['car_year_hand'].apply(deep_translate_text)
# df_car_prices['car_price_en'] = df_car_prices['car_price'].apply(deep_translate_text)
# df_car_prices['car_tags_en'] = df_car_prices['car_tag'].apply(deep_translate_text)

df_car_prices

TooManyRequests: Server Error: You made too many requests to the server.According to google, you are allowed to make 5 requests per secondand up to 200k requests per day. You can wait and try again later oryou can try the translate_batch function

In [ ]:
<div class="tag-list_upgradeItemFlagBox__qPadL"><span class="tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Test in force</font></font></span><span class="tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">first hand</font></font></span><span class="tag-list_tag__EX6gE flag_baseFlag__PIHA5 flag_grayFlag__1s_oj"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">without accidents</font></font></span></div>

SyntaxError: invalid syntax (1866410033.py, line 1)

In [85]:
df_car_prices.head()

,car_name,car_info,car_year_hand,car_price,car_tag,car_name_en
0,ג'יפ / Jeep רנגלר ארוך,4X4 Unlimited Sahara אוט' 5 דל' 2.0 (272 כ''ס),2021 • יד 1,"300,000 ₪","גלגלי מגנזיום,טסט בתוקף,יד ראשונה",Jeep / Jeep Wrangler long
1,אאודי Q7,4X4 Premium חשמלי דיזל אוט' 5 מק' 3.0 (374 כ''ס),2018 • יד 2,"216,000 ₪","חסכוני,היברידי,טסט בתוקף",Audi Q7
2,סובארו פורסטר,4X4 B אוט' 2.0 (150 כ''ס) [2008-2011],2011 • יד 3,"42,000 ₪","שטח קשוח,טסט בתוקף,מערכת מולטימדיה",Subaru Forester
3,ג'יפ / Jeep גרנד צ'ירוקי,4X4 Laredo אוט' 5 מק' 3.6 (286 כ''ס),2019 • יד 1,"160,000 ₪","בקרת שמירת מרחק,גלגלי מגנזיום,טסט בתוקף",Jeep / Jeep Grand Cherokee
4,טויוטה לנד קרוזר ארוך,"4X4 STD אוט' דיזל 5 מק' 3.0 (173 כ""ס)",2008 • יד 1,"123,000 ₪","מיכל דלק גדול,טסט בתוקף,יד ראשונה",Toyota Land Cruiser long


In [80]:
df_car_prices.to_csv('yad2_car_prices_1.csv', index=False)